# Web-scraping: сбор данных из баз данных и интернет-источников

*Алла Тамбовцева*

### Практикум 4. Введение в работу с API (на примере Википедии)

Если мы не используем готовые библиотеки для работы с API («оболочки» для построения запросов и обработки результатов), для работы достаточно импортировать уже знакомый нам модуль `requests`:

In [1]:
import requests

### Часть 1: получаем информацию о пользователях Википедии

### Задача 1

Создайте запрос для получения информации о пользователе Википедии *Ilya Voyager*, используя следующую информацию:

* ссылка на API как базу данных https://ru.wikipedia.org/w/api.php;
* действие (аргумент `action`) для получения результата – `query`;
* формат результата (аргумент `format`) – `json`;
* название списка с интересующей нас информацией (аргумент `list`) – `users`;
* имя пользователя (аргумент `ususers`) – `Ilya_Voyager`.

Строку с запросом сохраните в переменную `q1`. Отправьте запрос к API, используя модуль `requests`, сохраните результат в переменную `response1` и извлеките из полученного объекта ответ на запрос:

* в виде обычной строки (`.text`);
* в формате JSON (`.json()`).

Сравните полученные результаты.

In [2]:
q1 = "https://ru.wikipedia.org/w/api.php?action=query&format=json&list=users&ususers=Ilya_Voyager"
response1 = requests.get(q1)

In [3]:
# визуально одно и то же, но первое – строка, второе – словарь
# со словарем работать удобнее

print(response1.text)
print(response1.json())

{"batchcomplete":"","query":{"users":[{"userid":2403,"name":"Ilya Voyager"}]}}
{'batchcomplete': '', 'query': {'users': [{'userid': 2403, 'name': 'Ilya Voyager'}]}}


### Задача 2

Создайте запрос для получения информации по тому же участнику, только с уточнением разделов (полей), информация о которых нам нужна:

* аргумент `usprop`, разделы `gender` и `groups`.

Строку с запросом сохраните в переменную `q2`. Отправьте запрос к API и извлеките из полученного объекта ответ на запрос в формате JSON. Создайте список, состоящий из id участника, имени участника, пола участника и списка групп, к которым он принадлежит.  

In [4]:
q2 = "https://ru.wikipedia.org/w/api.php?action=query&format=json&list=users&ususers=Ilya_Voyager&usprop=gender|groups"

In [5]:
# .json() сразу после get()

res = requests.get(q2).json()
print(res)

{'batchcomplete': '', 'query': {'users': [{'userid': 2403, 'name': 'Ilya Voyager', 'groups': ['editor', 'rollbacker', 'uploader', '*', 'user', 'autoconfirmed'], 'gender': 'male'}]}}


In [6]:
# работаем как со словарем, внутри которого словари и списки
# последовательно извлекаем элементы

user = res["query"]["users"][0]
print(user)

{'userid': 2403, 'name': 'Ilya Voyager', 'groups': ['editor', 'rollbacker', 'uploader', '*', 'user', 'autoconfirmed'], 'gender': 'male'}


In [7]:
info = [user["userid"], user["name"], user["gender"], user["groups"]]
print(info)

[2403, 'Ilya Voyager', 'male', ['editor', 'rollbacker', 'uploader', '*', 'user', 'autoconfirmed']]


### Задача 3

Представьте, что нам нужно собрать информацию по нескольким участникам сразу. Известно, что имена пользователей можно вводить через `|`, как и названия разделов. Напишите код, который 

* забирает имена участников из списка `names`;
* забирает названия разделов из списка `fields`;
* подставляет всё в нужную строку с запросом, отправляет его к API и получает ответ в формате JSON. 

**Подсказка:** вспомните про форматирование строк и f-строки.

In [8]:
names = ["Ilya_Voyager", "Drbug", "Kaganer"]
fields = ["gender", "groups", "registration"]

In [9]:
# склеиваем строки в списке в одну строку с разделителями |

names_str = "|".join(names)
fields_str = "|".join(fields)

# подставляем в f-строку names_str и fields_str
req = f"https://ru.wikipedia.org/w/api.php?action=query&format=json&list=users&ususers={names_str}&usprop={fields_str}"

In [10]:
# отправляем запрос и забираем результат

users_dict = requests.get(req).json()
print(users_dict)

{'batchcomplete': '', 'query': {'users': [{'userid': 2403, 'name': 'Ilya Voyager', 'registration': None, 'groups': ['editor', 'rollbacker', 'uploader', '*', 'user', 'autoconfirmed'], 'gender': 'male'}, {'userid': 47, 'name': 'Drbug', 'registration': None, 'groups': ['*', 'user', 'autoconfirmed'], 'gender': 'unknown'}, {'userid': 1090, 'name': 'Kaganer', 'registration': None, 'groups': ['editor', 'filemover', 'rollbacker', '*', 'user', 'autoconfirmed'], 'gender': 'male'}]}}


### Задача 4

Форматировать строки не очень сложно, но если запросы сами по себе длинные, не всегда удобно. Поэтому для подстановки нужных аргументов можно использовать возможности модуля `requests`, а именно функции `get()`. Эта функция умеет объединять ссылку на API и пары *аргумент-значение*, если эти пары *аргумент-значение* поданы ей на вход в виде словаря. 

Решите задачу 3, пользуясь этой возможностью.

In [11]:
users_dict = requests.get(url = "https://ru.wikipedia.org/w/api.php", 
            params = {"action" : "query",
            "format" : "json", 
            "list" : "users", 
            "ususers" : names_str,
            "usprop" : fields_str}).json()
print(users_dict)

{'batchcomplete': '', 'query': {'users': [{'userid': 2403, 'name': 'Ilya Voyager', 'registration': None, 'groups': ['editor', 'rollbacker', 'uploader', '*', 'user', 'autoconfirmed'], 'gender': 'male'}, {'userid': 47, 'name': 'Drbug', 'registration': None, 'groups': ['*', 'user', 'autoconfirmed'], 'gender': 'unknown'}, {'userid': 1090, 'name': 'Kaganer', 'registration': None, 'groups': ['editor', 'filemover', 'rollbacker', '*', 'user', 'autoconfirmed'], 'gender': 'male'}]}}


### Задача 5

Превратите результаты, полученные в предыдущей задаче, в датафрейм pandas, где одна строка соответствует одному пользователю. Выгрузите результаты в файл Excel.

In [12]:
import pandas as pd

In [13]:
# важно – в датафрейм превращаем список словарей внутри users
# не сам большой словарь users_dict и не query,
# иначе в датафрейме будет одна строка и один столбец

df = pd.DataFrame(users_dict["query"]["users"])
df

,userid,name,registration,groups,gender
0,2403,Ilya Voyager,None,"[editor, rollbacker, uploader, *, user, autoco...",male
1,47,Drbug,None,"[*, user, autoconfirmed]",unknown
2,1090,Kaganer,None,"[editor, filemover, rollbacker, *, user, autoc...",male


In [14]:
df.to_excel("users_wiki.xlsx")

В завершение этой части – в самой документации к API есть готовые [примеры](https://www.mediawiki.org/wiki/API:Users) запросов на Python, причём с удобным способом их создания :)

### Часть 2: получаем текст статей Википедии

### Задача 6


В качестве примера выгрузки текста статьи в документации приведена страница об Альберте Эйнштейне. Попробуем выгрузить текст статьи по уже привычной схеме (только немного другие названия методов и аргументов):

In [15]:
URL = "https://en.wikipedia.org/w/api.php"

PARAMS = {
    "action": "parse",
    "page": "Albert Einstein",
    "prop": "wikitext",
    "format": "json"
}

R = requests.get(url=URL, params=PARAMS)
DATA = R.json()

Извлеките элемент с ключом `wikitext` и изучите его структуру. Удобно ли работать с текстом в таком виде?

In [16]:
# дикая строка с кучей скобок внутри 
# wikitext – текст с разметкой Википедии, свои специальные символы,
# даже с ходу не распарсишь

wikitext = DATA["parse"]["wikitext"]["*"]

### Задача 7

Измените в коде выше аргумент `prop` на `text`. Посмотрите, что получилось (по аналогии с предыдущей задачей нас интересует элемент `text`).

In [17]:
URL = "https://en.wikipedia.org/w/api.php"

# меняем prop
PARAMS = {
    "action": "parse",
    "page": "Albert Einstein",
    "prop": "text",
    "format": "json"
}

R = requests.get(url=URL, params=PARAMS)
DATA = R.json()

# повторяем – первые 200 символов для примера
# уже обычный HTML – хотя бы парсить будет проще

text = DATA["parse"]["text"]["*"]
print(text[0:200])

<div class="mw-content-ltr mw-parser-output" lang="en" dir="ltr"><div class="shortdescription nomobile noexcerpt noprint searchaux" style="display:none">German-born scientist (1879–1955)</div>
<style 


В свободном режиме давайте поизучаем, каким образом получить текст или его фрагменты в удобном виде.
Не помешает обратиться к документации по [методу](https://www.mediawiki.org/w/api.php?action=help&modules=parse) `parse` и по [методу](https://www.mediawiki.org/w/api.php?action=help&modules=query) `query`.

In [18]:
# ссылка та же

URL = "https://en.wikipedia.org/w/api.php"

# но выгружаем только вводную часть статьи – summary
# для многих задач хватает

# action = query, обычный запрос
# prop = extracts, отрывки
# exintro = True, забираем вводную часть
# explaintext = True, забираем отрывок (ex) в виде чистого текста (plaintext)
# titles, название статьи

PARAMS = {"action" : "query", 
         "prop" : "extracts", 
         "exintro" : True, 
         "explaintext" : True, 
         "titles" : "Albert Einstein",
         "format" : "json"}

In [19]:
# с этим уже можно работать

resp = requests.get(URL, PARAMS)
resp.json()

{'batchcomplete': '',
 'query': {'pages': {'736': {'pageid': 736,
    'ns': 0,
    'title': 'Albert Einstein',
    'extract': 'Albert Einstein ( EYEN-styne; German: [ˈalbɛɐt ˈʔaɪnʃtaɪn] ; 14 March 1879 – 18 April 1955) was a German-born theoretical physicist who is widely held to be one of the greatest and most influential scientists of all time. Best known for developing the theory of relativity, Einstein also made important contributions to quantum mechanics, and was thus a central figure in the revolutionary reshaping of the scientific understanding of nature that modern physics accomplished in the first decades of the twentieth century. His mass–energy equivalence formula E = mc2, which arises from relativity theory, has been called "the world\'s most famous equation". He received the 1921 Nobel Prize in Physics "for his services to theoretical physics, and especially for his discovery of the law of the photoelectric effect", a pivotal step in the development of quantum theory. His